# Adversarial Login Solver (TARGET_CLASS_ID = 101)

このノートブックは、`image.jpg` を微小摂動して MobileNetV2 がクラス ID 101 (tusker) と分類する敵対的画像を生成します。

- 前処理: MobileNetV2 の `preprocess_input` を使用
- 攻撃: ターゲット化 FGSM の反復適用 (勾配符号法)
- 制約: 元画像との MSE < 0.05 を維持
- 出力: `adversarial.jpg` と比較画像 `comparison.png`

最後にアプリの検証条件（MSEとクラスID）をセル内で再現して合格判定を表示します。


In [ ]:
import os
from PIL import Image
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# 定数（app.py と合わせる）
IMG_HEIGHT, IMG_WIDTH = 224, 224
TARGET_CLASS_ID = 101
MSE_THRESHOLD = 0.05
ORIGINAL_IMAGE_PATH = 'image.jpg'

print(tf.__version__)

# 前処理/後処理

def preprocess_image(img: Image.Image) -> tf.Tensor:
    img = img.convert('RGB').resize((IMG_HEIGHT, IMG_WIDTH))
    img_array = np.array(img)
    preprocessed = tf.keras.applications.mobilenet_v2.preprocess_input(img_array)
    return tf.convert_to_tensor(preprocessed, dtype=tf.float32)


def deprocess_image(img_np: np.ndarray) -> np.ndarray:
    img = img_np.copy().reshape((IMG_HEIGHT, IMG_WIDTH, 3))
    img = img + 1.0
    img = img * 127.5
    img = np.clip(img, 0, 255).astype(np.uint8)
    return img

# モデル読込
model = tf.keras.applications.MobileNetV2(weights='imagenet')

# 画像読込
orig_pil = Image.open(ORIGINAL_IMAGE_PATH)
orig_tensor = preprocess_image(orig_pil)

# 元画像の予測を確認
pred = model.predict(np.expand_dims(orig_tensor, axis=0), verbose=0)
print('Original top-1 class id:', int(np.argmax(pred[0])))
print('Original target prob:', float(pred[0][TARGET_CLASS_ID]))


In [ ]:
# ターゲット化FGSM（反復）
EPSILON = 0.03  # 摂動の大きさ（MobileNetV2前処理空間）
ITERATIONS = 200

adv = tf.Variable(orig_tensor, dtype=tf.float32)

best_img = None
best_prob = 0.0
best_mse = float('inf')

for i in range(ITERATIONS):
    with tf.GradientTape() as tape:
        tape.watch(adv)
        logits = model(tf.expand_dims(adv, 0))
        # ターゲットクラス確率を最大化
        loss = -logits[0, TARGET_CLASS_ID]
    grad = tape.gradient(loss, adv)
    signed = tf.sign(grad)
    adv.assign_sub(EPSILON * signed)

    # 監視
    probs = model.predict(tf.expand_dims(adv, 0), verbose=0)[0]
    prob = float(probs[TARGET_CLASS_ID])
    mse = float(tf.reduce_mean(tf.square(adv - orig_tensor)).numpy())
    if mse < MSE_THRESHOLD and prob > best_prob:
        best_img = adv.numpy().copy()
        best_prob = prob
        best_mse = mse
    if i % 10 == 0 or i == ITERATIONS - 1:
        print(f"iter={i:03d} prob={prob:.6f} mse={mse:.6f} pred={int(np.argmax(probs))}")

if best_img is None:
    best_img = adv.numpy()

adv_np = best_img
print('Best target prob:', best_prob)
print('Best MSE:', best_mse)


In [ ]:
# 検証・保存・可視化
from pathlib import Path

# 予測
final_probs = model.predict(np.expand_dims(adv_np, 0), verbose=0)[0]
final_pred = int(np.argmax(final_probs))
final_prob = float(final_probs[TARGET_CLASS_ID])

# MSE
mse = float(tf.reduce_mean(tf.square(adv_np - orig_tensor)).numpy())
print('Final pred:', final_pred, 'target prob:', final_prob, 'MSE:', mse)

# 可視化と保存
adv_img_uint8 = deprocess_image(adv_np)
Image.fromarray(adv_img_uint8).save('adversarial.jpg')
print("Saved: adversarial.jpg")

# 比較画像
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.title('Original')
plt.imshow(orig_pil)
plt.axis('off')

plt.subplot(1,2,2)
plt.title('Adversarial')
plt.imshow(adv_img_uint8)
plt.axis('off')
plt.tight_layout()
plt.savefig('comparison.png')
print('Saved: comparison.png')

# アプリの条件に合格か
print('\nMeets app condition:', (final_pred == TARGET_CLASS_ID and mse < MSE_THRESHOLD))
